In [92]:
### solution to puzzles described here http://adventofcode.com/2017/day/14

## Helper functions from Day 10 (hash knot)

def flip(vec, position, length):
    '''(list on int) -> list of int
    Return `vec` with elements position:(position+length) in reverse order'''
    len_original = len(vec)
    vec = 2 * vec
    arc_end = position + length
    arc = vec[position:arc_end]
    vec[position:arc_end] = reversed(arc)
    
    if arc_end >= len_original:
        spill = arc_end - len_original
        vec[0:spill] = vec[len_original:arc_end]
    
    return vec[:len_original]

def split_list(ll, sublist_length):
    '''(list, int) -> list of list
    Return `ll` split into pieces of length `sublist_length`'''
    starts = range(0, len(ll), sublist_length)
    return [ll[idx:(idx + sublist_length)] for idx in starts]

def collapse_xor(row):
    '''(list of int) -> int
    Return the result of reducing `row` by bitwise XOR.
    '''
    return reduce(lambda a, b: a ^ b, row)

def knot_hash(input_string):
    '''(str) -> str
    Return knot-hash of `input_string` as a 32-length string'''
    extra_chars = "".join(map(lambda x: chr(int(x)), '17, 31, 73, 47, 23'.split(',')))
    instructions = map(ord, input_string + extra_chars)
    vec = range(256)
    skip = 0
    position = 0

    for rr in range(64):
        for length in instructions:
            vec = flip(vec, position, length)
            position = (position + length + skip) % 256
            skip += 1

    folded_vec = split_list(vec, 16)
    list_of_dec = map(collapse_xor, folded_vec)
    list_of_hex = map(lambda y: "{0:02x}".format(y), list_of_dec)
    result_string = ''.join(list_of_hex)
    return result_string

## New helper functions
def dec2hex(x):
    '''(int) -> str
    Return a hexadecimal representation of `x`'''
    return format(x, 'x')

def dec2bin(x):
    '''(int) -> str
    Return a binary representation of `x`, 
    zero-padded to length 4'''
    return format(x, '04b')

# dictionary that will serve as a conversion function
# I'm too lazy lo look up the correct function for this
hex2bin = {dec2hex(x): dec2bin(x) for x in range(16)}

def knot_binary(input_string, hex2bin=hex2bin):
    '''(str, dict of str:str) -> str
    Return binary representaiton of knot hach of `input_string`.
    '''
    hex_hash = knot_hash(input_string)
    binary_list = [hex2bin[x] for x in list(hex_hash)]
    return ''.join(binary_list)

def get_grid(key):
    '''(str) -> list of str
    Return grid based on key as 128-length list of
    128-length binary strings.'''
    numbered_keys = [key + '-' + str(i) for i in range(128)]
    return map(knot_binary, numbered_keys)

def count_row(row):
    '''(str) -> int
    Return count of "1"s in `row`'''
    return sum(map(int, row))
    

In [94]:
## Read input

input_raw = 'hxtvlmkl'
test_input = 'flqrgnkx'
knot_binary(test_input + '-0')[:8]

'11010100'

In [144]:
##  Solution to PUZZLE 1
bin_grid = get_grid(input_raw)
result = sum(map(count_row, bin_grid))

print 'The answer to the first puzzle is', result

The answer to the first puzzle is 8214


In [184]:
##  Solution to PUZZLE 1

## More helper functions
def content(grid, row, col):
    '''(list of str, int, int) -> str
    Return content of grid at indices (`row`, `col`), 
    or "0" if out of bounds.'''
    if row < 0 or col < 0:
        return "0"
    
    try:
        result = grid[row][col]
    except IndexError:
        result = "0"
    return result

# this is not the best implementation, but it's simple
# a proper one would use a functional solution with 
# memory of visited positions and yield
def crawl(grid, row, col, count=0):
    '''(list of int, int, int) -> list of str, int
    Return `count` plus number of all adjacent "1" entries 
    starting from position (`row`, `col`). Modify grid in 
    place so counted "1" entries are changed to "2".
    '''
    if content(grid, row, col) != "1":
        return count
    else:
        old_row = list(grid[row])  ## hacky
        old_row[col] = "2"
        new_row = "".join(old_row)
        grid[row] = new_row
        right = crawl(grid, row + 1, col, count + 1)
        left = crawl(grid, row - 1, col, 0)
        up = crawl(grid, row, col - 1, 0)
        down = crawl(grid, row, col + 1, 0)
        return right + left + up + down
    
def count_blobs(grid):
    num_blobs = 0
    
    for row in range(len(grid)):
        for col in range(len(grid[row])):
            blob_size = crawl(grid, row, col)
            if blob_size > 0:
                num_blobs += 1
    return num_blobs

In [190]:
grid_copy = bin_grid[:]   ## deep copy
print "The answer to the second puzzle is", count_blobs(grid_copy)

The answer to the second puzzle is 1093
